# Building a Real-Time Search Agent with Linkup & Quotient

<a target="_blank" href="https://colab.research.google.com/github/quotient-ai/quotient-cookbooks/blob/main/cookbooks/search/linkup/linkup-quotient-detections.ipynb">
 <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This cookbook demonstrates how to build a real-time search agent using [Linkup](https://docs.linkup.so/) while monitoring result accuracy with [Quotient AI](https://www.quotientai.co/).

We'll cover:
- Using Linkup's API to perform real-time web search
- Processing search queries from a JSONL file
- Monitoring search result quality and detecting hallucinations with Quotient
- Understanding and improving the accuracy of search results


In [2]:
# install dependencies
! pip install -qU quotientai linkup-sdk tqdm

## Step 0: Grab your API keys

We'll use API keys from:
 - [Linkup](https://docs.linkup.so/) — get your API key from the [Linkup app](https://app.linkup.so/sign-up)
 - [Quotient AI](https://www.quotientai.co) — get your API key from the [Quotient AI app](https://app.quotientai.co)
 
Both Linkup and Quotient offer generous free tiers to get started.

In [ ]:
import os
# Set API keys:
os.environ['LINKUP_API_KEY'] = "your-linkup-api-key"
os.environ['QUOTIENT_API_KEY'] = "your-quotient-api-key"

## Step 1: Set up Linkup Search and Quotient Monitoring

We'll configure Linkup's API to perform real-time web search and use Quotient to monitor the quality of our search results. Our search queries will cover various topics.


In [4]:
from linkup import LinkupClient
from typing import Dict, Any
import json

# Initialize the client
client = LinkupClient(api_key=os.getenv('LINKUP_API_KEY'))

def get_search_results(query: str) -> Dict[str, Any]:
    """
    Get search results for a query using Linkup's sourcedAnswer.

    Args:
        query: The search query

    Returns:
        Dictionary containing the answer and source documents
    """
    try:
        # Clean input
        query = query.strip()
        
        # Call Linkup API
        response = client.search(
            query=query,
            depth="deep",  # Use deep for more thorough results
            output_type="sourcedAnswer"
        )
        
        # Access attributes directly from the LinkupSourcedAnswer object
        return {
            'answer': response.answer,
            'sources': response.sources
        }
        
    except Exception as e:
        return {
            'error': str(e),
            'query': query
        }


Quotient is an intelligent observability platform designed for retrieval-augmented and search-augmented AI systems.

Quotient performs automated detections on two key fronts each time you send it a log:

- **Hallucination:** Identifies statements in the model output that are unsupported by the retrieved documents or that contradict them. This flagging is done at the sentence level and returns a boolean indicator if any part of the answer contains a hallucination.

- **Document Relevance:** Evaluates each retrieved document to determine whether it meaningfully contributed to grounding the answer. Quotient returns relevance labels for all documents, helping gauge retrieval and search quality.
  
These capabilities are enabled automatically when `hallucination_detection=True` is set during logger initialization.

Below, we'll set up the Quotient logger, send each AI-search result for automatic evaluation, and retrieve structured logs and detections:


In [5]:
from quotientai import QuotientAI, DetectionType

# Initialize the Quotient SDK
quotient = QuotientAI()

logger = quotient.logger.init(
    # Name your application or project
    app_name="linkup-search-agent-demo",
    # Set the environment (e.g., "dev", "prod", "staging")
    environment="test",
    # Set the sample rate for logging (0-1.0)
    sample_rate=1.0,
    # this will automatically run hallucination detection on 100% of your model outputs in relation to the documents you provide
    detections=[DetectionType.HALLUCINATION, DetectionType.DOCUMENT_RELEVANCY],
    detection_sample_rate=1.0,
)


## Step 2: Load and Process Search Queries

We'll load search queries from a JSONL file that contains various real-world questions. Each query will be processed to:
1. Get relevant search results from Linkup
2. Monitor result quality with Quotient
3. Track performance metrics

In [ ]:
def load_queries(file_path: str, max_queries: int = 100) -> list:
    """
    Load queries from a JSONL file.
    
    Args:
        file_path: Path to the JSONL file
        max_queries: Maximum number of queries to load
        
    Returns:
        List of query strings
    """
    queries = []
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            if i >= max_queries:
                break
            query_obj = json.loads(line)
            queries.append(query_obj['query'])
    return queries

# Load example queries
queries = load_queries('example_queries.jsonl', max_queries=120)
print(f"Loaded {len(queries)} queries. First 3 queries:")
for i, query in enumerate(queries[:3]):
    print(f"{i+1}. {query}")

Loaded 140 queries. First 3 queries:
1. What’s the difference between GPT-4o and Claude 3?
2. Who is the CEO of Anthropic?
3. Summarize today’s top stories on climate change.


Let's generate detailed, structured descriptions for each company using Linkup's API. For each company, we will:
1. Generate a comprehensive search query
2. Use Linkup's structured output feature to extract company information
3. Format the results according to our schema
4. Log the results in Quotient for quality monitoring

The structured output ensures consistent formatting and makes it easy to use the company information in downstream applications.


In [ ]:
log_ids = []

for i, query in enumerate(queries, 1):
    print(f"\nProcessing query {i}/{len(queries)}: {query}")
    result = get_search_results(query)
    
    if 'error' in result:
        print(f"❌ Error: {result['error']}")
        continue
        
    print(f"\n📊 Search Results:\n{result['answer']}")
    
    # Log to Quotient for quality monitoring
    log_id = quotient.log(
        user_query=query,
        model_output=result['answer'],
        documents=[str(doc) for doc in result['sources']]
    )
    
    print(f"📝 Logged to Quotient with log_id: {log_id}")
    log_ids.append(log_id)



Processing query 1/140: What do people say about the Rabbit R1 AI device?

📊 Search Results:
People's opinions about the Rabbit R1 AI device are mixed but generally critical:

- Many reviewers find the device underwhelming and "half-baked," with features that often don't work as promised or are limited in functionality. (WIRED, Gizmodo, The Shortcut)
- The scroll wheel and physical interface receive criticism for being inconsistent and frustrating to use. (WIRED)
- Users question the need for a separate device when smartphones can perform similar tasks, making the R1 feel redundant and inconvenient to carry. (Reddit r/ArtificialIntelligence, r/hardware)
- Battery life and software bugs were issues at launch, though some improvements have been made via updates. (The Shortcut)
- The device is praised for its design and concept, with some seeing it as a glimpse into the future of AI interaction, but many feel it is not ready for mainstream use. (Pixel Refresh, Reddit r/Rabbitr1)
- The La

### How It Works

When `.log()` is called:

1. **Data ingestion:** The query, model output, and all retrieved document contents are logged to Quotient.

2. **Async detection pipeline:** Quotient runs:
- **Hallucination detection**, labeling the output as hallucinated or not.
- **Document relevance scoring**, marking which retrieved documents helped ground the output 

3. **Result retrieval:** You can poll or fetch detections linked to your `log_id`.

4. **Monitor and troubleshoot in the Quotient app:** Access the [Quotient dashboard](app.quotientai.co) to:
- Monitor your AI system over time
- Review flagged hallucinated sentences
- See which documents were irrelevant
- Compare across tags or environments for deeper insights

For full implementation details, visit the Quotient [docs](https://docs.quotientai.co/).


# Step 4: Review `Detections` and `Reports` in Quotient

You can view your logs and detections in the [Quotient dashboard](app.quotientai.co), where you can filter them by tags and environments to identify common failure patterns.

![Quotient AI Dashboard](Quotient_Dashboard.png "Quotient AI Dashboard")

## 🚨 New: Daily Reports and Query Analysis

Quotient now provides automated daily reports that help you understand your search agent's performance at scale. 

![Quotient Reports](Quotient_Reports.png "Quotient AI Reports")

The new `Reports` tab offers:

### 1. Semantic Query Clustering
- Automatically groups similar queries together
- Shows you what topics your users are most interested in
- Helps identify patterns in search behavior

### 2. Topic-Level Analytics
- Hallucination rates and Document Relevance scores per topic cluster
- Identifies which topics need the most attention

### 3. Risk Assessment
- Ranks issues by both volume and severity
- Highlights high-risk areas that need immediate attention
- Tracks performance trends over time
  
![Quotient Reports](Quotient_Reports_2.png "Quotient AI Reports")

### Using Reports for Agent Improvement

The Reports feature is particularly valuable for:
1. **Content Gap Analysis**: Identify topics where your search agent consistently struggles
2. **Query Optimization**: Find patterns in queries that lead to poor results
3. **Resource Allocation**: Focus improvements on high-volume or high-risk areas
4. **Trend Monitoring**: Track how system performance changes over time

To access these insights, visit the `Reports` tab in your Quotient dashboard after running 100+ queries.

## What You've Built

A production-ready search agent that:
- Takes natural language queries as input
- Returns comprehensive, well-researched answers
- Provides source attribution for fact verification
- Monitors accuracy through Quotient's hallucination detection
- Verifies information quality with document relevance scoring

You can scale this to monitor production traffic, benchmark retrieval and search performance, or compare different models side by side.


## Understanding Result Quality

When evaluating the company descriptions, pay attention to these metrics:

- **Hallucination Rate**: Should be **< 5%**
  - Higher rates might indicate:
    - Outdated or conflicting company information online
    - Need for more specific company identifiers in queries
    - Issues with source document quality

- **Document Relevance**: Should be **> 75%**
  - Lower scores might suggest:
    - Company name ambiguity (e.g., common names or multiple companies)
    - Need for additional company identifiers (e.g., industry, location)
    - Source documents discussing different companies or topics
